# Lab 3: Policy Search
Solution written by Serra Matteo and Matteo Magnaldi.

In [12]:
import logging
from functools import reduce
from math import inf
from copy import deepcopy
import random
from functools import cache

In [13]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self.safe = self.evaluate()
        self.winner = 0  
    
    
    def niming(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects
        if sum(self._rows) == 0:
            logging.info("Yeuch!")

    def evaluate(self) -> bool:
        safe = reduce((lambda x,y: x ^ y), self._rows)
        self._safe = safe == 0
        return self._safe
    
    def is_finished(self) -> bool:
        return all(map(lambda x: x == 0, self._rows))
    
    def __hash__(self):
        return tuple(self._rows).__hash__()

def minCost(arr):
     
    cost = inf
    elem = -1
    # calculate XOR sum of array
    XOR = 0
    for e in arr:
         XOR ^= e
 
    # find the min cost and element
    # corresponding
    r_cost = 0
    for i,e in enumerate(arr):
        new_cost = abs((XOR ^ e) - e)
        if (cost > new_cost):
            r_cost = ((XOR ^ e) - e)
            if ((XOR ^ e) - e) > 0: 
                continue
            else:
                cost = (XOR ^ e) - e
            element = e
 
    #print("Element = ", element)
    #print("Operation required = ", r_cost)
    return arr.index(element),-r_cost


def random_move(nim:Nim):
    rand_row = random.randint(0,len(nim._rows)-1)
    while nim._rows[rand_row] == 0:
        rand_row = random.randint(0,len(nim._rows)-1)

    if nim._rows[rand_row] > nim._k:
        rand_numObj = random.randint(1,nim._k)
    else:
        rand_numObj = random.randint(1,nim._rows[rand_row])

    return rand_row,rand_numObj

def nonZeroRows(nim:Nim):
    return len(nim._rows) - [n == 0 for n in nim._rows].count(True)

def expert_system(nim: Nim) -> (int, int):
    # when a single row counts more than an object, invert the strategy 
    invert_strategy = [n > 1 for n in nim._rows].count(True) == 1 and max(nim._rows) <= nim._k
    safe= nim.evaluate()
    if all([n == 1 or n == 0 for n in nim._rows]):
        return random_move(nim)
    else:
        if invert_strategy:
            #when the number of rows whith one remain element will be odd, remove n elements to have all rows with one
            maxElem = max(nim._rows)  
            if nonZeroRows(nim) % 2 != 0 :
                return nim._rows.index(maxElem),maxElem-1
            else:
                return nim._rows.index(maxElem),maxElem
        else:
            if safe:
                # random move or something more clever
                #return random_move(nim)
                #cerca una mossa per cui l'avversario non possa mandarti in sicuro
                for i,n in enumerate(nim._rows):
                    for k in range(1,nim._k+1):
                        if n<k:
                            continue
                        temp_nim = nim._rows[:]
                        temp_nim[i] -= k
                        if minCost(temp_nim)[1] > nim._k:
                            #print("trovata")
                            return i,k
                return random_move(nim)
            else:
                # comeback to safe
                modRows = [row % (nim._k+1) for row in nim._rows ]
                return minCost(modRows)

def randomBanchmark(nIter,n,k,p1_strategy,p2_strategy):
    v = 0
    for i in range(nIter):
        nim = Nim(n,k)
        v += play(nim,p1_strategy,p2_strategy)
    print(f"winner ratio: {v/nIter*100}") 


def play(nim: Nim,p1_strategy,p2_strategy):
    random.seed()
    turn = random.randint(0,1)
    #print(f"turn:{'player'if not turn else 'random'}")
    while not nim.is_finished():
        if turn:
            row, k = p2_strategy(nim)
        else:
            row,k = p1_strategy(nim)
        #print(f"move of {'player'if not turn else 'random'}, row:{row} remove {k} elements")
        turn = not turn
        nim.niming(row,k)
        #print(nim._rows)
    #print(f"The winner is {'player'if not turn else 'random'}")
    return int(not turn)

def possible_actions(nim: Nim):
    
    for i,r in enumerate(nim._rows):
        for e in range(nim._k, 0, -1):
            if r >= e:     
                ret = deepcopy(nim)
                ret.niming(i, e)
                print(f"{ret._rows}")
                yield ret, (i, e)

# primo int: riga da cui rimuovere, secondo int: quanti elementi rimuovere
@cache
def minmax(state, isMaximizingPlayer, alpha=-1, beta=1, max_depth = 0, ):
    current_depth = 0
    moves = []
    if state.is_finished():
        state.winner = 1 if isMaximizingPlayer else -1
        return state.winner, moves
    #print(f"evaluating state: {state._rows}")
    
    if isMaximizingPlayer:
        best = -inf
        for child, move in possible_actions(state):
            moves.append(move)
            print(f"depth: {max_depth}, alpha: {alpha}, beta: {beta}, moves: {moves}")
            
            value, old_moves = minmax(child, False, alpha, beta, current_depth+1)
            if(best < value):
                pass
            else:
                moves.pop()

            best = max(best, value)
            alpha = max(alpha, best)
            if beta <= alpha:
                break
        return best, moves
    else:
        best = +inf
        for child, move in possible_actions(state):
            #print(f"{move}")
            moves.append(move)
            print(f"depth: {max_depth}, alpha: {alpha}, beta: {beta}, moves: {moves}")
            
            value, old_moves = minmax(child, True, alpha, beta, current_depth+1)
            if(best > value):
                pass
            else:
                moves.pop()
            best = min(best, value)
            beta = min(beta, best)

            if beta <= alpha:
                break
        return best, moves






## Expert System

In [15]:
randomBanchmark(10000,8,6,expert_system,random_move)
randomBanchmark(10000,8,6,random_move,random_move)


KeyboardInterrupt: 

In [ ]:
nim = Nim(2, 3)
minmax(nim, True, -inf, inf)
# for n in possible_actions(nim):
#     print(f"{n._rows}"



[0, 3]
depth: 0, alpha: -inf, beta: inf, moves: [(0, 1)]
[0, 0]
depth: 1, alpha: -inf, beta: inf, moves: [(1, 3)]
[0, 1]
depth: 1, alpha: -inf, beta: 1, moves: [(1, 3), (1, 2)]
[0, 0]
depth: 1, alpha: -inf, beta: 1, moves: [(1, 1)]
[0, 2]
depth: 1, alpha: -inf, beta: -1, moves: [(1, 3), (1, 2), (1, 1)]
[0, 0]
depth: 1, alpha: -inf, beta: -1, moves: [(1, 2)]
[1, 0]
depth: 0, alpha: -1, beta: inf, moves: [(0, 1), (1, 3)]
[0, 0]
depth: 1, alpha: -1, beta: inf, moves: [(0, 1)]
[1, 1]
depth: 0, alpha: 1, beta: inf, moves: [(0, 1), (1, 3), (1, 2)]
[0, 1]
depth: 1, alpha: 1, beta: inf, moves: [(0, 1)]
[0, 0]
depth: 1, alpha: 1, beta: inf, moves: [(1, 1)]
[1, 2]
depth: 0, alpha: 1, beta: inf, moves: [(0, 1), (1, 3), (1, 1)]
[0, 2]
depth: 1, alpha: 1, beta: inf, moves: [(0, 1)]
[0, 0]
depth: 1, alpha: 1, beta: inf, moves: [(1, 2)]
[0, 1]
depth: 1, alpha: 1, beta: inf, moves: [(1, 2), (1, 1)]
[0, 0]
depth: 1, alpha: 1, beta: inf, moves: [(1, 1)]


(1, [(0, 1), (1, 3)])